In [1]:
%cd C:/Users/gioc4/Documents/python-benchmark

C:\Users\gioc4\Documents\python-benchmark


In [2]:
import pandas as pd
import statsmodels.api as sm
from provider_scorecard.benchmark import Benchmark

# load data
df = pd.read_csv("C:/Users/gioc4/Documents/blog/data/sparcs2.zip")
 
# transform
# get only the top 30 drg codes
df = df[df['Hospital County'] == 'Manhattan']

# some recoding
df['Length of Stay'] = [120 if x == '120 +' else x for x in df['Length of Stay']]
df['Length of Stay'] = df['Length of Stay'].astype(int)
df['APR DRG Code'] = df['APR DRG Code'].astype(str)

# top 30 drg codes
drg = df.groupby('APR DRG Code')['APR DRG Code'].count().sort_values(ascending=False).head(30)
drg = drg.astype(str)

# final dataframe
df = df[df['APR DRG Code'].isin(drg.index)]

C:\Users\gioc4\AppData\Local\Temp\ipykernel_18484\3824810403.py:6: DtypeWarning: Columns (10,29) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("C:/Users/gioc4/Documents/blog/data/sparcs2.zip")


In [3]:
df.groupby('Facility Name')['Facility Name'].count().sort_values(ascending=False)

Facility Name
Mount Sinai Hospital                                              26687
NYU Langone Hospitals                                             22156
New York-Presbyterian Hospital - New York Weill Cornell Center    21582
New York-Presbyterian Hospital - Columbia Presbyterian Center     18491
Lenox Hill Hospital                                               17679
Mount Sinai West                                                  13179
Bellevue Hospital Center                                          11563
Hospital for Special Surgery                                       8534
New York Presbyterian Hospital - Allen Hospital                    6803
Mount Sinai Morningside                                            6741
New York-Presbyterian/Lower Manhattan Hospital                     5688
Memorial Hospital for Cancer and Allied Diseases                   5393
Metropolitan Hospital Center                                       4403
Mount Sinai Beth Israel                           

In [4]:
# input features
xfeat = [
    "Age Group",
    "Gender",
    "Race",
    "Length of Stay",
    "Type of Admission",
    "APR DRG Code",
    "APR Severity of Illness Description",
    "APR Risk of Mortality",
]

# evaluation features
efeat = ["Total Charges", "Total Costs", "Payment Typology 1"]

# focal hospitals
hosp = [
    'Mount Sinai Hospital',
    'NYU Langone Hospitals',
    'Lenox Hill Hospital'
]

In [5]:
res_list = []
mod_list = []

# loop through hospitals, store dataframes in list
for h in hosp:
    tr = df["Facility Name"] == h

    bch = Benchmark(
        data=df, focal_indicator=tr, predictor_features=xfeat, evaluation_features=efeat
    )
    bch.fit(lrate=0.1, nest = 500, **{'tree_method':'hist', 'alpha': 0, 'lambda': 1})
    bch.evaluate()

    # store results
    mod_list.append(bch)

    # extract coefficients
    b = [item[0] for item in bch.outcomes]
    se = [item[1] for item in bch.outcomes]
    res_list.append(
        pd.DataFrame(
            {"Hospital": h, "Outcome": bch.Xeval.columns, "Value": b,"SE":se}
        )
    )

c:\Users\gioc4\Anaconda3\envs\benchmark\Lib\site-packages\xgboost\data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
c:\Users\gioc4\Anaconda3\envs\benchmark\Lib\site-packages\xgboost\data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
c:\Users\gioc4\Anaconda3\envs\benchmark\Lib\site-packages\xgboost\data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
c:\Users\gioc4\Anaconda3\envs\benchmark\Lib\site-packages\xgboost\data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return is

In [6]:
# concat into single pandas dataframe
results = pd.concat(res_list, axis=0)

In [7]:
results

,Hospital,Outcome,Value,SE
0,Mount Sinai Hospital,Total Charges,-12178.46,283.735015
1,Mount Sinai Hospital,Total Costs,-7114.35,94.654466
2,Mount Sinai Hospital,Payment Typology 1_Blue Cross/Blue Shield,-0.02,0.001488
3,Mount Sinai Hospital,Payment Typology 1_Federal/State/Local/VA,-0.00,0.000089
4,Mount Sinai Hospital,"Payment Typology 1_Managed Care, Unspecified",0.01,0.000389
5,Mount Sinai Hospital,Payment Typology 1_Medicaid,0.02,0.001912
6,Mount Sinai Hospital,Payment Typology 1_Medicare,0.01,0.001247
7,Mount Sinai Hospital,Payment Typology 1_Miscellaneous/Other,-0.00,0.000308
8,Mount Sinai Hospital,Payment Typology 1_Private Health Insurance,-0.02,0.001974
9,Mount Sinai Hospital,Payment Typology 1_Self-Pay,0.00,0.000428
